In [ ]:
"""

- Imoveis na periferia de São Paulo são subvalorizados?

- A ideia que norteia o projeto é entender se os imoveis que em raio de 30km do marco 0 da cidade de São Paulo são subvalorizados em comparação ao centro ou bairros nobres
- Para possibilitar uma analise de recomendação de investimento nesses imoveis com base em expectativa de lucro, tempo de retorno e analise de risco do mercado.
- Possiveis interessados na analise exploratoria: Startups do mercado imobiliario, Fundos Imobiliarios, Construtoras, Instituições financeiras que fornecem credito imobiliario

- analise baseada nos imoveis disponiveis nos sites Loft, Quinto Andar e Zap Imoveis

- a primeira missão seria conseguir as informações dos imoveis listados com [id_anuncio, endereco, preco, m2, quartos, banheiro, vaga de garagem, tipo imovel, novo/usado, rating atribuido por mim]
- se possivel obter alguma informação sobre a venda desses imoveis (talvez diario oficial para buscar tranferencias de imoveis)
- com base no endereço começamos a classificar se o imovel está em area periferica, neutra ou nobre
- pós classificação começamos a fazer as comparações de rating e preco de imoveis similares em locais diferentes
- tirar mais ideias sobre a situação imobiliaria de sao paulo
- tentar criar algoritmo que busca oportunidades de lucro 

-- linkar com problemas habitacionais da cidade
-- se possivel realizar uma regressao para comparar o quanto cada variavel impacta no preço do m2/total do imovel e nao se esquecer de adicionar uma variavel para segurança do bairro e outros fatores externos


WebScreaping 
Beautiful soup
Seleniun
Paralelização 

--quando fizer o webscraping procurar as <tags> que contenha apenas a info que eu quero
--para buscar diversas tags diferentes é interessante criar uma função com condicionais para cada type de informação dentro da tag

EXEMPLO
tags = ['p','h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7']
text_list = []
for element in soup.find_all(tags):
    text_list.append(element.text)
print('\n'.join(text_list))


"""

In [ ]:
"""TASK LIST 
- fazer o scraping de uma das variaveis do imovel em uma pagina simples [feito]
- criar uma lista dos links dos imoveis da tela principal



"""

# Imports e acesso ao banco de dados

In [1]:
# Imports que vou utilizar no codigo

from tqdm import tqdm
import pandas as pd
import numpy as np

import requests
from datetime import datetime

from bs4 import BeautifulSoup
from lxml import etree

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys



In [2]:
#localização do chromewebdriver
chrome_path = r'C:\Users\T-GAMER\Desktop\Ironhack DA\Projeto Final\chromedriver'
ser = Service(chrome_path)

In [3]:
#chromewebdriver deve estar aberto para o codigo funcionar
driver = webdriver.Chrome(service=ser)

# Base coleta de dados primeira pagina

In [ ]:
url = 'https://loft.com.br/imovel/apartamento-rua-francisco-rossano-vila-prudente-sao-paulo-1-quarto-40m2/ycf79w'

In [ ]:
driver.get(url)

In [ ]:
driver.current_url

In [ ]:
#metodo do webdriver 'find_element para procurar um elemento dentro da minha URL pelo xpath' 
#peguei o caminho full_xpath do valor no grande box a direita
driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[2]/div/div/div/div/div/div/ul/li[2]/p').text

In [ ]:
#peguei o caminho full_xpath do ID do Imóvel nas letras cinzas embaixo das fotos
#será necessário limpar o DataFrame para criar uma chave unica deste ID
driver.find_element('xpath', '/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[1]/nav/ol/li[9]/span').text

In [ ]:
#peguei o caminho full_xpath da rua nas letras cinzas embaixo das fotos 
driver.find_element('xpath', '/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[1]/nav/ol/li[7]/a').text

In [ ]:
#peguei o caminho full_xpath do tipo do imovel nas descrições em preto embaixo das fotos 
#será necessário limpar o tipo com regex, seria uma boa ideia entender os tipos de imovel dentro do site e depois filtrar eles
driver.find_element('xpath', '/html/body/div[1]/section/div[2]/div[1]/div[3]/div[2]/div[1]/p').text

In [ ]:
#peguei o caminho full_xpath do titulo nas descrições em preto embaixo das foto para extrair a quantidade de quartos, suites e vagas de gaaragem  
#será necessário limpar o tipo com regex
driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[2]/h1').text

In [ ]:
#peguei o caminho full_xpath das descrições para extrair a quantidade de banheiros.  
#será necessário limpar o tipo com regex
driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[1]/div[3]/div[2]/div[5]/p').text

# Func de Coleta uma pagina unica

In [4]:
url = 'https://loft.com.br/imovel/apartamento-rua-francisco-rossano-vila-prudente-sao-paulo-1-quarto-40m2/ycf79w'
driver.get(url)

In [36]:

df_list = []

def coleta_info(x):
    
    """Essa funcao coleta os dados de um imovel na pagina de anuncio do site Loft por meio do Xpath e salva estes dados dentro de um DataFrame
    Atualmente para a funcao ser executada corretamente é necessario carregar a pagina previamente no navegador de maneira manual
    Eventualmente o site atualiza de um dia para o outro e os caminhos de Xpath devem ser atualizados"""
    
    url = x
    driver.get(url)
    dict_teste = {}
    
    
    try:
        dict_teste['preco_imov'] = driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[2]/div/div/div/div/div/div/ul/li[2]/p').text
    except:
        dict_teste['preco_imov'] = 'Indisponivel'     
    try:    
        dict_teste['id_imov'] = driver.find_element('xpath', '/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[1]/nav/ol/li[9]/span').text
    except:
        dict_teste['id_imov'] = 'Indisponivel'       
    try:
        dict_teste['cidade'] = driver.find_element('xpath', '/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[1]/nav/ol/li[3]/a').text
    except: 
        dict_teste['cidade'] = 'Indisponivel'      
    try:    
        dict_teste['endereco'] = driver.find_element('xpath', '/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[1]/nav/ol/li[7]/a').text
    except: 
        dict_teste['endereco'] = 'Indisponivel'  
    try:    
        dict_teste['titulo_imov'] = driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[2]/h1').text
    except: 
        dict_teste['titulo_imov'] = 'Indisponivel'
    try:    
        dict_teste['vaga'] = driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[1]/div[3]/div[2]/div[3]/p').text
    except: 
        dict_teste['vaga'] = 'Indisponivel'
    try:    
        dict_teste['banheiro'] = driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[1]/div[3]/div[2]/div[4]/p').text
    except: 
        dict_teste['banheiro'] = 'Indisponivel'
    try:    
        dict_teste['condominio'] = driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[2]/div/div/div/div/div/div/ul/div[2]/li[1]/div[2]/p').text
    except:
        dict_teste['condominio'] = 'Indisponivel'
    try:
        dict_teste['iptu'] = driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[2]/div/div/div/div/div/div/ul/div[2]/li[2]/div[2]/p').text
    except: 
        dict_teste['iptu'] = 'Indisponivel'
        
    return df_list.append(pd.DataFrame(dict_teste, index=[0]))
    
    #global df_teste
    #df_teste = pd.DataFrame(dict_teste, index=[0]) 
    
    #Quando eu tiro o Global e deixo ele apenas retornando o DataFrame o valor nao sai da função pra nenhum lugar #
    

In [18]:
coleta_info('https://loft.com.br/imovel/apartamento-rua-sao-luis-santana-porto-alegre-3-quartos-234m2/1rjt5zo')

In [6]:
df_list

[]

# Automatização do codigo para rodar em varias paginas.

In [5]:
# coleta de lista de links da home do site
# inicia na 2 e vai até a 132

home_url = f'https://loft.com.br/venda/imoveis/sp/sao-paulo?pagina={i}'
home_response = requests.get(home_url)
html = home_response.content
soup = BeautifulSoup(html)

# List comprehantion que junta o url da home filtrando dentro da tag <a> todas as tags <href> que iniciam em '/imovel'

lista_links = ['https://loft.com.br' + tag ['href'] for tag in soup.find_all('a') if tag['href'].startswith('/imovel')]


NameError: name 'i' is not defined

In [6]:
# For que cria uma lista de todos os links de imoveis listados para sao paulo
# Lista extraida no dia 08/09/2022

lista_links = []

for i in range(1,132):
    home_url = f'https://loft.com.br/venda/imoveis/sp/sao-paulo?pagina={i}'
    home_response = requests.get(home_url)
    html = home_response.content
    soup = BeautifulSoup(html)
    lista_links += ['https://loft.com.br' + tag ['href'] for tag in soup.find_all('a') if tag['href'].startswith('/imovel')]



In [7]:
len(lista_links)

4978

In [5]:
# For que cria uma lista de todos os links de imoveis listados para porto alegre
# Lista extraida no dia 09/09/2022

lista_links_pa = []

for i in range(1,132):
    home_url = f'https://loft.com.br/venda/imoveis/rs/porto-alegre?pagina={i}'
    home_response = requests.get(home_url)
    html = home_response.content
    soup = BeautifulSoup(html)
    lista_links_pa += ['https://loft.com.br' + tag ['href'] for tag in soup.find_all('a') if tag['href'].startswith('/imovel')]

In [6]:
len(lista_links_pa)

4978

In [8]:
# For que cria uma lista de todos os links de imoveis listados para rio de janeiro
# Lista extraida no dia 09/09/2022

lista_links_rj = []

for i in range(1,132):
    home_url = f'https://loft.com.br/venda/imoveis/rj/rio-de-janeiro?pagina={i}'
    home_response = requests.get(home_url)
    html = home_response.content
    soup = BeautifulSoup(html)
    lista_links_rj += ['https://loft.com.br' + tag ['href'] for tag in soup.find_all('a') if tag['href'].startswith('/imovel')]

In [9]:
len(lista_links_rj)

4978

In [33]:
# For que cria uma lista de todos os links de imoveis listados para rio de janeiro
# Lista extraida no dia 09/09/2022

lista_links_bh = []

for i in range(1,132):
    home_url = f'https://loft.com.br/venda/imoveis/mg/belo-horizonte?pagina={i}'
    home_response = requests.get(home_url)
    html = home_response.content
    soup = BeautifulSoup(html)
    lista_links_bh += ['https://loft.com.br' + tag ['href'] for tag in soup.find_all('a') if tag['href'].startswith('/imovel')]

In [34]:
len(lista_links_bh)

4978

In [45]:
lista_teste = lista_links_bh [4001:4977]

In [46]:
# Funcao para coletar informacoes de cada imovel dentro da lista de links, estou rodando de 100 em 100 imoveis para evitar um block do site por ip ou um timeout
# após rodar o list(map) tenho que rodar o concat para incluir as informações recem coletadas no df_final

list(map(coleta_info, tqdm(lista_teste)))


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 976/976 [49:32<00:00,  3.05s/it]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [102]:
df_final_sp = pd.concat(df_list)
df_final_sp

,preco_imov,id_imov,cidade,endereco,titulo_imov,vaga,banheiro,etc_apart,condominio,iptu
0,R$ 1.484.000,Imóvel 1RUQ6KB,Apartamento em São Paulo,Rua dos Pinheiros,"Apartamento à venda em Pinheiros com 68 m², 2 ...",1 vaga,2 Banheiros,Lavabo,R$ 530/mês,R$ 1.830/ano
0,R$ 1.200.000,Imóvel 1NDUYA8,Apartamento em São Paulo,Rua Camburiú,Apartamento à venda em Alto da Lapa com 115 m²...,2 vagas,2 Banheiros,Posição da unidade,R$ 1.195/mês,R$ 391/mês
0,R$ 1.160.000,Imóvel 1UT6HT2,Apartamento em São Paulo,Rua Cunha Gago,"Apartamento à venda em Pinheiros com 60 m², 2 ...",1 vaga,2 Banheiros,Indisponivel,R$ 984/mês,R$ 276/mês
0,R$ 580.000,Imóvel NGVFTG,Apartamento em São Paulo,Rua Adelino de Almeida Castilho,"Apartamento à venda em Tatuapé com 68 m², 2 qu...",1 vaga,2 Banheiros,Posição da unidade,R$ 400/mês,Isento
0,R$ 795.000,Imóvel WBLJ5Z,Apartamento em São Paulo,Rua Araçoiaba,"Apartamento à venda em Saúde com 64 m², 2 quar...",1 vaga,2 Banheiros,Indisponivel,R$ 791/mês,R$ 3.200/ano
...,...,...,...,...,...,...,...,...,...,...
0,R$ 550.000,Imóvel 1MU2S8P,Apartamento em São Paulo,Rua Ausonia,"Apartamento à venda em Tucuruvi com 60 m², 2 q...",2 vagas,2 Banheiros,Indisponivel,R$ 450/mês,R$ 100/ano
0,R$ 355.000,Imóvel NAM32A,Apartamento em São Paulo,Rua Campevas,"Apartamento à venda em Perdizes com 32 m², 1 q...",1 vaga,1 Banheiro,Chuveiro,R$ 630/mês,R$ 160/mês
0,R$ 850.000,Imóvel 5L45RB,Apartamento em São Paulo,Avenida Portugal,"Apartamento à venda em Brooklin com 58 m², 1 q...",1 vaga,1 Banheiro,Lavabo,R$ 1.000/mês,R$ 230/mês
0,R$ 529.000,Imóvel 1UJRSYM,Apartamento em São Paulo,Rua Dona Ana Neri,"Apartamento à venda em Cambuci com 62 m², 3 qu...",1 vaga,2 Banheiros,Vista emparedada,R$ 550/mês,R$ 64/mês


In [117]:
df_final_sp.to_csv(r"C:\Users\T-GAMER\Desktop\Ironhack DA\Projeto Final\raw_dataset.csv", encoding = 'latin1')

In [29]:
df_final_pa = pd.concat(df_list)
df_final_pa

,preco_imov,id_imov,cidade,endereco,titulo_imov,vaga,banheiro,condominio,iptu
0,R$ 535.000,Imóvel PEXGS5,Apartamento em Porto Alegre,Avenida Assis Brasil,"Apartamento à venda em Niterói com 72 m², 3 qu...",3 quartos\nsem suíte,1 Banheiro,R$ 420/mês,Isento
0,R$ 180.000,Imóvel S08XSE,Apartamento em Porto Alegre,Rua Professor Cristiano Fischer,"Apartamento à venda em Jardim Salso com 62 m²,...",2 quartos\nsem suíte,1 vaga,R$ 240/mês,R$ 66/ano
0,R$ 1.100.000,Imóvel 1RJT5ZO,Apartamento em Porto Alegre,Rua São Luís,"Apartamento à venda em Santana com 234 m², 3 q...",3 quartos\n1 suíte,2 vagas,R$ 537/mês,R$ 2.912/ano
0,R$ 350.000,Imóvel HB6LN8,Apartamento em Porto Alegre,Rua Ari Marinho,"Apartamento à venda em Higienópolis com 68 m²,...",2 quartos\nsem suíte,1 Banheiro,R$ 300/mês,R$ 33/mês
0,R$ 125.000,Imóvel VVX91J,Apartamento em Porto Alegre,Rua Orfanotrófio,"Apartamento à venda em Santa Tereza com 49 m²,...",2 quartos\nsem suíte,1 vaga,R$ 230/mês,R$ 230/ano
...,...,...,...,...,...,...,...,...,...
0,R$ 368.000,Imóvel 19W9WS8,Apartamento em Porto Alegre,Rua Pedro Américo,"Apartamento à venda em São João com 92 m², 3 q...",3 quartos\nsem suíte,1 Banheiro,R$ 300/mês,Isento
0,R$ 233.700,Imóvel 1O90Y32,Apartamento em Porto Alegre,Rua Luiz Afonso,"Apartamento à venda em Cidade Baixa com 42 m²,...",1 quarto\nsem suíte,1 Banheiro,R$ 250/mês,R$ 100/mês
0,R$ 1.250.000,Imóvel 1M4ZNT4,Apartamento em Porto Alegre,Rua Monroe,Apartamento à venda em Santa Tereza com 287 m²...,3 quartos\n1 suíte,2 vagas,R$ 960/mês,R$ 1.200/ano
0,R$ 807.000,Imóvel 1L6UV84,Apartamento em Porto Alegre,Rua Comendador Rodolfo Gomes,"Apartamento à venda em Menino Deus com 92 m², ...",3 quartos\n1 suíte,1 vaga,R$ 1.000/mês,R$ 2.500/mês


In [33]:
df_final_pa.to_csv(r"C:\Users\T-GAMER\Desktop\Ironhack DA\Projeto Final\raw_dataset_pa.csv", encoding = 'latin1')

In [31]:
df_final_rj = pd.concat(df_list)
df_final_rj

,preco_imov,id_imov,cidade,endereco,titulo_imov,vaga,banheiro,condominio,iptu
0,R$ 340.000,Imóvel E11NI0,Apartamento em Rio de Janeiro,Avenida Dom Hélder Câmara,"Apartamento à venda em Penha com 49 m², 2 quar...",2 quartos\nsem suíte,1 vaga,R$ 413/mês,R$ 240/ano
0,R$ 1.150.000,Imóvel R37EZV,Apartamento em Rio de Janeiro,Rua Otaviano Hudson,"Apartamento à venda em Copacabana com 82 m², 3...",3 quartos\n1 suíte,1 vaga,R$ 1.212/mês,R$ 307/mês
0,R$ 1.200.000,Imóvel MOIQAX,Apartamento em Rio de Janeiro,Rua Paissandu,"Apartamento à venda em Laranjeiras com 85 m², ...",3 quartos\nsem suíte,1 vaga,R$ 1.600/mês,R$ 2.143/ano
0,R$ 480.000,Imóvel QOZQAQ,Apartamento em Rio de Janeiro,Avenida Prado Júnior,"Apartamento à venda em Copacabana com 30 m², 1...",1 quarto\nsem suíte,1 Banheiro,R$ 615/mês,R$ 82/mês
0,R$ 1.350.000,Imóvel 1Q2R4LP,Apartamento em Rio de Janeiro,Rua Pereira da Silva,"Apartamento à venda em Laranjeiras com 90 m², ...",3 quartos\n1 suíte,1 vaga,R$ 1.400/mês,R$ 1.994/ano
...,...,...,...,...,...,...,...,...,...
0,R$ 1.200.000,Imóvel 1KXD2SR,Apartamento em Rio de Janeiro,Avenida Carlos Peixoto,"Apartamento à venda em Botafogo com 94 m², 3 q...",3 quartos\n1 suíte,1 vaga,R$ 1.962/mês,R$ 359/mês
0,R$ 1.150.000,Imóvel 13P4EDA,Apartamento em Rio de Janeiro,Rua Figueiredo Magalhães,"Apartamento à venda em Copacabana com 136 m², ...",3 quartos\n1 suíte,1 vaga,R$ 1.600/mês,Isento
0,R$ 1.440.000,Imóvel 15OKJYS,Apartamento em Rio de Janeiro,Rua Inhanga,"Apartamento à venda em Copacabana com 220 m², ...",4 quartos\n1 suíte,4 Banheiros,R$ 2.600/mês,R$ 632/mês
0,R$ 3.600.000,Imóvel WPG5ZI,Apartamento em Rio de Janeiro,Avenida Atlântica,"Apartamento à venda em Copacabana com 240 m², ...",4 quartos\n4 suítes,1 vaga,R$ 3.450/mês,R$ 1.010/mês


In [32]:
df_final_rj.to_csv(r"C:\Users\T-GAMER\Desktop\Ironhack DA\Projeto Final\raw_dataset_rj.csv", encoding = 'latin1')

In [47]:
df_final_bh = pd.concat(df_list)
df_final_bh

,preco_imov,id_imov,cidade,endereco,titulo_imov,vaga,banheiro,condominio,iptu
0,R$ 390.000,Imóvel 122XXGY,Apartamento em Belo Horizonte,Rua José Mendes de Carvalho,"Apartamento à venda em Castelo com 115 m², 4 q...",4 quartos\n1 suíte,2 vagas,R$ 325/mês,R$ 1.260/ano
0,R$ 330.000,Imóvel PV674P,Apartamento em Belo Horizonte,Rua Geraldo Faria de Souza,Apartamento à venda em Sagrada Família com 106...,3 quartos\n1 suíte,1 vaga,R$ 330/mês,R$ 133/ano
0,R$ 340.000,Imóvel H0JAZ5,Apartamento em Belo Horizonte,Rua Castelo de São Jorge,"Apartamento à venda em Castelo com 75 m², 3 qu...",3 quartos\n1 suíte,2 vagas,R$ 380/mês,R$ 145/ano
0,R$ 1.190.000,Imóvel B817P6,Apartamento em Belo Horizonte,Rua Boa Esperança,"Apartamento à venda em Sion com 150 m², 4 quar...",4 quartos\n1 suíte,2 vagas,R$ 700/mês,R$ 5.786/ano
0,R$ 480.000,Imóvel 14D84PD,Apartamento em Belo Horizonte,Rua Ulisses Marcondes Escobar,"Apartamento à venda em Buritis com 100 m², 3 q...",3 quartos\n1 suíte,2 vagas,R$ 550/mês,R$ 177/ano
...,...,...,...,...,...,...,...,...,...
0,R$ 217.000,Imóvel LSESEF,Apartamento em Belo Horizonte,Rua das Chácaras,"Apartamento à venda em Maria Helena com 47 m²,...",2 quartos\nsem suíte,1 vaga,R$ 288/mês,R$ 45/mês
0,R$ 240.000,Imóvel COA60K,Apartamento em Belo Horizonte,Rua W Quatro,"Apartamento à venda em Pongelupe com 68 m², 3 ...",3 quartos\nsem suíte,1 vaga,R$ 95/mês,Isento
0,R$ 220.000,Imóvel 1UB7Y4V,Apartamento em Belo Horizonte,Rua Cachoeira da Prata,"Apartamento à venda em Pindorama com 44 m², 2 ...",2 quartos\nsem suíte,1 vaga,R$ 350/mês,R$ 99/ano
0,R$ 150.000,Imóvel 1S1LTP8,Apartamento em Belo Horizonte,Rua Monte Real,Apartamento à venda em Granja de Freitas com 4...,2 quartos\nsem suíte,1 vaga,R$ 50/mês,Isento


In [48]:
df_final_bh.to_csv(r"C:\Users\T-GAMER\Desktop\Ironhack DA\Projeto Final\raw_dataset_bh.csv", encoding = 'latin1')

# Data Cleaning

In [ ]:
# importar todos os CSVs e junta-los em um unico Data Frame
# criar a coluna de distancia do marco 0 de SP em um raio de 30km
# segregar em zonas para facilitar analiuse 

In [74]:
df_clean = pd.read_csv(r"C:\Users\T-GAMER\Desktop\Ironhack DA\Projeto Final\raw_dataset.csv", encoding = 'latin1')

In [ ]:
pd.merge

In [8]:
df_sp = pd.read_csv('raw_dataset_sp.csv', encoding = 'latin1')
df_pa = pd.read_csv('raw_dataset_pa.csv', encoding = 'latin1')
df_rj = pd.read_csv('raw_dataset_rj.csv', encoding = 'latin1')
df_bh = pd.read_csv('raw_dataset_bh.csv', encoding = 'latin1')

In [27]:
df_sp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4968 entries, 0 to 4967
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   preco_imov   4968 non-null   object
 1   id_imov      4968 non-null   object
 2   cidade       4968 non-null   object
 3   endereco     4968 non-null   object
 4   titulo_imov  4968 non-null   object
 5   vaga         4968 non-null   object
 6   banheiro     4968 non-null   object
 7   condominio   4968 non-null   object
 8   iptu         4968 non-null   object
dtypes: object(9)
memory usage: 349.4+ KB


In [28]:
df_pa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4975 entries, 0 to 4974
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   4975 non-null   int64 
 1   preco_imov   4975 non-null   object
 2   id_imov      4975 non-null   object
 3   cidade       4975 non-null   object
 4   endereco     4975 non-null   object
 5   titulo_imov  4975 non-null   object
 6   vaga         4975 non-null   object
 7   banheiro     4975 non-null   object
 8   condominio   4975 non-null   object
 9   iptu         4975 non-null   object
dtypes: int64(1), object(9)
memory usage: 388.8+ KB


In [29]:
df_rj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4974 entries, 0 to 4973
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   4974 non-null   int64 
 1   preco_imov   4974 non-null   object
 2   id_imov      4974 non-null   object
 3   cidade       4974 non-null   object
 4   endereco     4974 non-null   object
 5   titulo_imov  4974 non-null   object
 6   vaga         4974 non-null   object
 7   banheiro     4974 non-null   object
 8   condominio   4974 non-null   object
 9   iptu         4974 non-null   object
dtypes: int64(1), object(9)
memory usage: 388.7+ KB


In [30]:
df_bh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4975 entries, 0 to 4974
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   4975 non-null   int64 
 1   preco_imov   4975 non-null   object
 2   id_imov      4975 non-null   object
 3   cidade       4975 non-null   object
 4   endereco     4975 non-null   object
 5   titulo_imov  4975 non-null   object
 6   vaga         4975 non-null   object
 7   banheiro     4975 non-null   object
 8   condominio   4975 non-null   object
 9   iptu         4975 non-null   object
dtypes: int64(1), object(9)
memory usage: 388.8+ KB


In [ ]:
df_total.info()

In [10]:
df_sp.drop(columns= ['Unnamed: 0', 'etc_apart'], inplace= True)

In [31]:
df_total = pd.concat([df_sp, df_pa, df_rj, df_bh])

In [32]:
df_total

,preco_imov,id_imov,cidade,endereco,titulo_imov,vaga,banheiro,condominio,iptu,Unnamed: 0
0,R$ 1.484.000,Imóvel 1RUQ6KB,Apartamento em São Paulo,Rua dos Pinheiros,"Apartamento à venda em Pinheiros com 68 m², 2 ...",1 vaga,2 Banheiros,R$ 530/mês,R$ 1.830/ano,NaN
1,R$ 1.200.000,Imóvel 1NDUYA8,Apartamento em São Paulo,Rua Camburiú,Apartamento à venda em Alto da Lapa com 115 m²...,2 vagas,2 Banheiros,R$ 1.195/mês,R$ 391/mês,NaN
2,R$ 1.160.000,Imóvel 1UT6HT2,Apartamento em São Paulo,Rua Cunha Gago,"Apartamento à venda em Pinheiros com 60 m², 2 ...",1 vaga,2 Banheiros,R$ 984/mês,R$ 276/mês,NaN
3,R$ 580.000,Imóvel NGVFTG,Apartamento em São Paulo,Rua Adelino de Almeida Castilho,"Apartamento à venda em Tatuapé com 68 m², 2 qu...",1 vaga,2 Banheiros,R$ 400/mês,Isento,NaN
4,R$ 795.000,Imóvel WBLJ5Z,Apartamento em São Paulo,Rua Araçoiaba,"Apartamento à venda em Saúde com 64 m², 2 quar...",1 vaga,2 Banheiros,R$ 791/mês,R$ 3.200/ano,NaN
...,...,...,...,...,...,...,...,...,...,...
4970,R$ 217.000,Imóvel LSESEF,Apartamento em Belo Horizonte,Rua das Chácaras,"Apartamento à venda em Maria Helena com 47 m²,...",2 quartos\nsem suíte,1 vaga,R$ 288/mês,R$ 45/mês,0.0
4971,R$ 240.000,Imóvel COA60K,Apartamento em Belo Horizonte,Rua W Quatro,"Apartamento à venda em Pongelupe com 68 m², 3 ...",3 quartos\nsem suíte,1 vaga,R$ 95/mês,Isento,0.0
4972,R$ 220.000,Imóvel 1UB7Y4V,Apartamento em Belo Horizonte,Rua Cachoeira da Prata,"Apartamento à venda em Pindorama com 44 m², 2 ...",2 quartos\nsem suíte,1 vaga,R$ 350/mês,R$ 99/ano,0.0
4973,R$ 150.000,Imóvel 1S1LTP8,Apartamento em Belo Horizonte,Rua Monte Real,Apartamento à venda em Granja de Freitas com 4...,2 quartos\nsem suíte,1 vaga,R$ 50/mês,Isento,0.0


In [33]:
df_total.drop(columns='Unnamed: 0', inplace=True)

In [42]:
df_total.drop_duplicates(inplace=True)

In [43]:
df_total

,preco_imov,id_imov,cidade,endereco,titulo_imov,vaga,banheiro,condominio,iptu
0,R$ 1.484.000,Imóvel 1RUQ6KB,Apartamento em São Paulo,Rua dos Pinheiros,"Apartamento à venda em Pinheiros com 68 m², 2 ...",1 vaga,2 Banheiros,R$ 530/mês,R$ 1.830/ano
1,R$ 1.200.000,Imóvel 1NDUYA8,Apartamento em São Paulo,Rua Camburiú,Apartamento à venda em Alto da Lapa com 115 m²...,2 vagas,2 Banheiros,R$ 1.195/mês,R$ 391/mês
2,R$ 1.160.000,Imóvel 1UT6HT2,Apartamento em São Paulo,Rua Cunha Gago,"Apartamento à venda em Pinheiros com 60 m², 2 ...",1 vaga,2 Banheiros,R$ 984/mês,R$ 276/mês
3,R$ 580.000,Imóvel NGVFTG,Apartamento em São Paulo,Rua Adelino de Almeida Castilho,"Apartamento à venda em Tatuapé com 68 m², 2 qu...",1 vaga,2 Banheiros,R$ 400/mês,Isento
4,R$ 795.000,Imóvel WBLJ5Z,Apartamento em São Paulo,Rua Araçoiaba,"Apartamento à venda em Saúde com 64 m², 2 quar...",1 vaga,2 Banheiros,R$ 791/mês,R$ 3.200/ano
...,...,...,...,...,...,...,...,...,...
4969,R$ 318.651,Imóvel WWQJ51,Apartamento em Belo Horizonte,Rua Outeiro,"Apartamento à venda em Jatobá com 112 m², 2 qu...",2 quartos\nsem suíte,1 vaga,R$ 200/mês,R$ 150/mês
4970,R$ 217.000,Imóvel LSESEF,Apartamento em Belo Horizonte,Rua das Chácaras,"Apartamento à venda em Maria Helena com 47 m²,...",2 quartos\nsem suíte,1 vaga,R$ 288/mês,R$ 45/mês
4972,R$ 220.000,Imóvel 1UB7Y4V,Apartamento em Belo Horizonte,Rua Cachoeira da Prata,"Apartamento à venda em Pindorama com 44 m², 2 ...",2 quartos\nsem suíte,1 vaga,R$ 350/mês,R$ 99/ano
4973,R$ 150.000,Imóvel 1S1LTP8,Apartamento em Belo Horizonte,Rua Monte Real,Apartamento à venda em Granja de Freitas com 4...,2 quartos\nsem suíte,1 vaga,R$ 50/mês,Isento


In [62]:
df_total.to_csv('df_total.csv', encoding='latin1')

In [76]:
df_clean ['id_imov'] = df_clean['id_imov'].str.replace('Imóvel','', regex = True)

In [77]:
df_clean ['cidade'] = df_clean ['cidade'].str.replace('Apartamento em ', '', regex = True)

In [78]:
df_clean ['metros'] = df_clean ['titulo_imov'].str.findall('\d+ m²')
#('\d+ m²', '', regex = True)

In [84]:
df_clean.drop(columns = 'Unnamed: 0', inplace=True)

In [106]:
df_clean ['teste'] = df_clean ['titulo_imov'].str.replace('\d+', '', regex = True)

In [108]:
df_clean ['teste'] = df_clean ['teste'].str.replace('(Apartamento)+', '', regex = True)

In [110]:
df_clean ['teste'] = df_clean ['teste'].str.replace('(venda em)+', '', regex = True)

In [112]:
df_clean ['teste'] = df_clean ['teste'].str.replace('(com)+', '', regex = True)

In [114]:
df_clean ['teste'] = df_clean ['teste'].str.replace('(à)+', '', regex = True)

In [118]:
df_clean ['teste'] = df_clean ['teste'].str.replace('m²,', '', regex = True)

In [122]:
df_clean ['teste'] = df_clean ['teste'].str.replace('quarto,', '', regex = True)

In [130]:
df_clean ['teste'] = df_clean ['teste'].str.replace('suítes,', '', regex = True)

In [132]:
df_clean ['bairro'] = df_clean ['teste']

In [135]:
df_clean.to_excel(r"C:\Users\T-GAMER\Desktop\Ironhack DA\Projeto Final\clean_dataset_sp.xlsx", encoding = 'latin1')